Below is an example of the sift algorithm, with the trilinear interpolation smoothing of a histogram tensor of the generate descriptors being implemented in hardware using our overlay, histogram_tensor

This overlay stream in inputs for the histogram tensor calculations, which are done on the board, and the resulting structure is stream back out and placed back into the software structure for further calculations

In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/descriptor/design_1.bit')
dma = overlay.axi_dma_0
# dma?

In [ ]:
from sfm_algo_unpacked import localizeExtremumViaQuadraticFit, computeKeypointsWithOrientations, isPixelAnExtremum
from numpy import floor

def findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width, contrast_threshold=0.04):
    threshold = floor(0.5 * contrast_threshold / num_intervals * 255)
    keypoints = []

    for octave_index, dog_images_in_octave in enumerate(dog_images):
        for image_index, (first_image, second_image, third_image) in enumerate(zip(dog_images_in_octave, dog_images_in_octave[1:], dog_images_in_octave[2:])):
            for i in range(image_border_width, first_image.shape[0] - image_border_width):
                if (i % 10 == 0):
                    print('.', end='')
                if (i % 100 == 0):
                    print('')
                for j in range(image_border_width, first_image.shape[1] - image_border_width):
                    if isPixelAnExtremum(first_image[i-1:i+2, j-1:j+2], second_image[i-1:i+2, j-1:j+2], third_image[i-1:i+2, j-1:j+2], threshold):
                        localization_result = localizeExtremumViaQuadraticFit(i, j, image_index + 1, octave_index, num_intervals, dog_images_in_octave, sigma, contrast_threshold, image_border_width)
                        if localization_result is not None:
                            keypoint, localized_image_index = localization_result
                            keypoints_with_orientations = computeKeypointsWithOrientations(keypoint, octave_index, gaussian_images[octave_index][localized_image_index])
                            for keypoint_with_orientation in keypoints_with_orientations:
                                keypoints.append(keypoint_with_orientation)
    return keypoints

In [ ]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors, generateDescriptors_hardware
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2



img1 = cv2.imread('dino_data/dino02.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('dino_data/dino05.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
print("Finished keypoints")
keypoints = removeDuplicateKeypoints(keypoints)
print("Finished remove duplicate keypoints")
kp1 = convertKeypointsToInputImageSize(keypoints)
print("Finished convert to image size")
des1 = generateDescriptors_hardware(keypoints, gaussian_images, dma)
print("Finished generate_descriptors")

base_image = generateBaseImage(img2, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
print("Finished keypoints")
keypoints = removeDuplicateKeypoints(keypoints)
print("Finished remove duplicate keypoints")
kp2 = convertKeypointsToInputImageSize(keypoints)
print("Finished convert to image size")
des2 = generateDescriptors_hardware(keypoints, gaussian_images, dma)
print("Finished generate_descriptors")

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)


# Estimate homography between template and scene
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

M = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

# Draw detected template in scene image
h, w = img1.shape
pts = np.float32([[0, 0],
                    [0, h - 1],
                    [w - 1, h - 1],
                    [w - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, M)

img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

h1, w1 = img1.shape
h2, w2 = img2.shape
nWidth = w1 + w2
nHeight = max(h1, h2)
hdif = int((h2 - h1) / 2)
newimg = np.zeros((nHeight, nWidth, 3), np.uint8)

for i in range(3):
    newimg[hdif:hdif + h1, :w1, i] = img1
    newimg[:h2, w1:w1 + w2, i] = img2

# Draw SIFT keypoint matches
for m in good:
    pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1] + hdif))
    pt2 = (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1]))
    cv2.line(newimg, pt1, pt2, (255, 0, 0))

plt.imshow(newimg)
plt.show()

In [ ]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors, generateDescriptors_hardware
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

img1 = cv2.imread('dino_data/dino02.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
# keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
# print("Finished keypoints")
# keypoints = removeDuplicateKeypoints(keypoints)
# print("Finished remove duplicate keypoints")

In [ ]:
# Converts a KeyPoint into a Dictionary
def convertKPToDict(KP):
    return {
        'angle': KP.angle,
        'class_id': KP.class_id,
        'octave': KP.octave,
        'point': KP.pt,
        'response': KP.response,
        'size': KP.size
    }

def convertDictToKP(D):
    return cv2.KeyPoint(*D['point'],
                   D['size'],
                   D['angle'],
                   D['response'],
                   D['octave'])

def KeyPointsToJSON(KPs):
    output_list = []
    for KP in KPs:
        output_list.append(convertKPToDict(KP))
    return output_list

def unpackOctave(keypoint):
    """Compute octave, layer, and scale from a keypoint
    """
    octave = 5308915 & 255
    layer = (5308915 >> 8) & 255
    if octave >= 128:
        octave = octave | -128
    scale = 1 / np.float32(1 << octave) if octave >= 0 else np.float32(1 << -octave)
    return octave, layer, scale


In [ ]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors, generateDescriptors_hardware
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import cv2


print("Finished convert to image size")
keypoints_json = []
with open('data.json', 'r') as f:
    pre_keypoints_json = json.load(f)
    for kp in pre_keypoints_json:
        keypoints_json.append(convertDictToKP(kp))
print(len(keypoints_json))
print(len(gaussian_images))
print(unpackOctave(2818547))
# print(len(keypoints))
# kp1 = convertKeypointsToInputImageSize(keypoints)
des1 = generateDescriptors_hardware(keypoints_json, gaussian_images, dma)
print("Finished generate_descriptors")

In [ ]:
img2 = cv2.imread('dino_data/dino05.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
print("Finished keypoints")
keypoints = removeDuplicateKeypoints(keypoints)
print("Finished remove duplicate keypoints")

In [ ]:
kp2 = convertKeypointsToInputImageSize(keypoints)
print("Finished convert to image size")
des2 = generateDescriptors_hardware(keypoints, gaussian_images, dma)
print("Finished generate_descriptors")

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)


# Estimate homography between template and scene
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

M = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

# Draw detected template in scene image
h, w = img1.shape
pts = np.float32([[0, 0],
                    [0, h - 1],
                    [w - 1, h - 1],
                    [w - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, M)

img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

h1, w1 = img1.shape
h2, w2 = img2.shape
nWidth = w1 + w2
nHeight = max(h1, h2)
hdif = int((h2 - h1) / 2)
newimg = np.zeros((nHeight, nWidth, 3), np.uint8)

for i in range(3):
    newimg[hdif:hdif + h1, :w1, i] = img1
    newimg[:h2, w1:w1 + w2, i] = img2

# Draw SIFT keypoint matches
for m in good:
    pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1] + hdif))
    pt2 = (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1]))
    cv2.line(newimg, pt1, pt2, (255, 0, 0))

plt.imshow(newimg)
plt.show()